In [359]:
from vk_api import *
import json
from math import *
import time
import pandas as pd
import requests
import re
import pickle as pk

never_constant = time.time()


class VkParser():
    # может создать из логина и пароля, либо из файла с логин паролем
    def __init__(self, passfile = None, login = None, password = None):
        if (passfile is not None):
            with open(passfile, encoding = "utf8") as in_file:
                data = in_file.read()
                logins = json.loads(data)
                login, password = logins[0]

        self.ErrorCode = None
        self.wall_collect = 0

        self.session = VkApi(login, password, api_version = "5.89")
        try:
            self.session.auth(token_only = True)
        except AuthError as e:
            print("500" + str(e))
            self.ErrorCode = 1

        self.api = self.session.get_api()
        self.tools = VkTools(self.session)
        
    def format_addresses(self, addresses):
        """Возвращает из htt...com/xxx  xxx для всех адресов из списка"""
        res = []
        for page in addresses:
            temp = page[page.rfind("/") + 1 : ]
            if (temp[0:2] != "id" and temp.isnumeric()):
                temp = "id" + temp
            res.append(temp)
        print("Formating completed")
        return res

    def collect_easy_features(self, page_ids = None):
        """собирает простые признаки,
        то есть выкачивает их из vk"""
        res = []
        for i in range(0, len(page_ids), 1000):
            res.extend(self.collect_easy_features_1000(page_ids[i : i + 1000]))
        return res
        
    def get_creation_date(self, pg_id):
        r = requests.get(url = "https://vk.com/foaf.php", params = {"id": pg_id})
        ee = re.search(':created dc:date="([\d, T, \+, \-, :]*)"', r.text)
        if (ee is None):
            return never_constant
        s = ee.group(1)
        tm = (int(i) for i in [s[0:4], s[5:7], s[8:10], s[11:13], s[14:16], s[17:19], 0, 0, 0])
        tm = tuple(tm)
        return time.mktime(tm)
    
    # принимает имя, либо id страницы и собирает фичи, возвращает словарь
    def collect_easy_features_1000(self, page_ids):
        # см vk.com/dev/objects/user
        print("Start Collecting 1000")
        #state 0 - страница забанена или удалена, 1 - страница закрыта
        #2 не собирается признак стены, 3 - все отлично
        f2 = []
        
        text_field_names = ["about", "activities", "books", "games", "interests", "music", "movies", "quotes", "tv"] 
        pg_opts = ",".join(["city", "connections", "contacts", "counters", "crop_photo", "domain", \
                "education", "folowers_count", "last_seen", "music", "movies", \
                "photo_max_orig", "relatives", "relation", "schools",\
                "screen_name", "status", "universities", "verified"])
        
        with VkRequestsPool(self.session) as pool:
            for i, page_id in enumerate(page_ids):
                f2.append(pool.method("users.get", {"user_ids": page_id, "fields": pg_opts}))
        
        f1 = []
        for i in range(len(f2)):
            if (f2[i].ok == False or len(f2[i].result) < 1):
                print("!!!! Err " + str(page_ids[i]) + " !!! " + str(f2[i].error))                                
            else:
                f1.append(f2[i].result[0])
        
        print("Main info collected")
        
        for pg in f1:
            pg["creation_date"] = self.get_creation_date(pg["id"])
        
        print("Dates collected")
        
        if (self.wall_collect != 0):
            print("Photo Collection")
            with VkRequestsPool(self.session) as pool:
                for pg in f1:
                    if ("deactivated" in pg):
                        continue
                    if ("photos" in pg["counters"]):
                        photos_num = pg["counters"]["photos"]
                        pg["first_photo_date"] = pool.method("photos.getAll", 
                                                             {"owner_id": pg["id"], "offset": photos_num - 1})

            for pg in f1:
                if ("deactivated" in pg):
                        continue
                if ("photos" in pg["counters"] and pg["counters"]["photos"] > 0):
                    pg["first_photo_date"] = pg["first_photo_date"].result["items"][0]["date"]
                else:
                    pg["first_photo_date"] = never_constant
            
            print("Wall Collection")
            with VkRequestsPool(self.session) as pool:
                for pg in f1:
                    if ("deactivated" not in pg and (pg["is_closed"] == False or pg["can_access_closed"] == True)):
                        pg["wall_records"] = pool.method(method = "wall.get", 
                                                         values = {"owner_id": pg["id"], "count": 100})
            
            print("First Records collected")
            
            for pg in f1:
                if ("deactivated" not in pg and (pg["is_closed"] == False or pg["can_access_closed"] == True)):
                    if (pg["wall_records"].ok):
                        pg["wall_records"] = pg["wall_records"].result
                        pg["counters"]["posts"] = pg["wall_records"]["count"]
                        pg["state"] = 3
                        pg["wall_records"] = pg["wall_records"]["items"]
                        if (len(pg["wall_records"]) > 0):
                            pg["first_post_date"] = pg["wall_records"][-1]["date"]
                        else:
                            pg["first_post_date"] = never_constant
                        
                    else:
                        print(pg["wall_records"].error)
                        pg["state"] = 5
        
        for pg in f1:
            if ("deactivated" in pg):
                pg["state"] = 0
            elif (pg["is_closed"] == True and pg["can_access_closed"] == False):
                pg["state"] = 1
            elif("state" not in pg):
                pg["state"] = 2
            
            # self.tools.get_all("photos.getAlbums", 2, {owner_id : page_id})
        print("Collection Completed 1000")
        
        
        return f1
        
    def extract_easy_features(self, f1):
        """'count_av'
        'count_con ссылки на другие соц сети
        'count_fr количество друзей
        'count_gr количество групп
        'count_pg количество подписок
        'count_status количество стоп слов в статус
        'created когда создана
        'created_diff разница между первой фоткой и первой записью
        'f_to_f отношение подписчиков к друзьям
        'name_changed изменено ли имя страницы
        'photo_crop обрезана ли фотография
        'photo_exist есть ли фотография
        'state удалена - 0, закрыта - 1, не анализируется стена и дата фоток - 2, 3 - все ок, 5 - произошла боль (лимит)
        'text_field_counter - сколько непустых полей (1 это оч плохо)
        'wall_likefriends отношение лайков к друзьям * 10
        'wall_likeview отношение лайков к просмотрам * 10
        'wall_median медиана разницы времени постинга
        
        Обработка фич, можно сказать часть нормализации"""
        
        print("Started extraction")
        
        res = []
        for pg in f1:
            res.append({})
            cur = res[-1]
            cur["state"] = pg["state"]
            cur["id"] = pg["id"]
            cur["created"] = pg["creation_date"]
            if (pg["state"] == 1 or pg["state"] == 2 or pg["state"] == 3):
                # страница закрыта
                cur["created_ago"] = time.time() - cur["created"]
                cur["screen_name"] = pg["screen_name"]
                if ("groups" in pg["counters"]):
                    cur["count_gr"] = pg["counters"]["groups"]
                else:
                    cur["count_gr"] = 0
                if ("pages" in pg["counters"]):
                    cur["count_gr"] += pg["counters"]["pages"]
                if ("subscriptions" in pg["counters"]):
                    cur["count_gr"] += pg["counters"]["subscriptions"]
                
                if ("posts" in pg["counters"]):
                    cur["count_wa"] = pg["counters"]["posts"]
                elif ("wall_records" in pg):
                    cur["count_wa"] = len(pg["wall_records"])
                else:
                    cur["count_wa"] = -1
                if ("friends" in pg["counters"]):
                    cur["count_fr"] = pg["counters"]["friends"]
                else:
                    cur["count_fr"] = -1
                if ("crop_photo" in pg):
                    temp = pg["crop_photo"]["crop"]
                    cur["photo_crop"] = int(abs(temp["x"] - 0) > 5) + int(abs(temp["y"] - 0) > 5) + \
                                        int(abs(temp["x2"] - 100) > 5) + int(abs(temp["y2"] - 100) > 5)
                else:
                    cur["photo_crop"] = 0
                if ("photos" in pg["counters"]):
                    cur["photo_exist"] = pg["counters"]["photos"] > 0
                else:
                    cur["photo_exist"] = 0
                cur["count_status"] = 5
                if ("status" in pg):
                    for i in ["com", "http", "ru", "org", "www"]:
                        if (i in pg["status"]):
                            cur["count_status"] -= 1
                if (pg["screen_name"][0:2] != "id"):
                    cur["name_changed"] = 1
                else:
                    cur["name_changed"] = 0
                            
            
            if (pg["state"] == 2 or pg["state"] == 3):                
                # отношение количества подписчиков к количеству друзей
                
                if ("first_photo_date" not in pg):
                    pg["first_photo_date"] = never_constant
                if ("first_post_date" not in pg):
                    pg["first_post_date"] = -never_constant
                
                cur["created_diff"] = min(abs(pg["first_photo_date"] - pg["first_post_date"]),
                                          abs(pg["first_photo_date"] - pg["creation_date"]),
                                          abs(pg["first_post_date"] - pg["creation_date"]))
                
                if ("followers" in pg["counters"] and "friends" in pg["counters"] and pg["counters"]["friends"] > 0):
                    cur["folow_to_friends"] = pg["counters"]["followers"] / pg["counters"]["friends"]
                else:
                    cur["folow_to_friends"] = 1.0
                cur["count_av"] = 0
                if ("audios" in pg["counters"]):
                    cur["count_av"] += pg["counters"]["audios"]
                if ("videos" in pg["counters"]):
                    cur["count_av"] += pg["counters"]["videos"] * 2
                cur["count_con"] = 0
                # инста просто часто используется для рекламы
                for i in ["skype", "facebook", "twitter", "livejournal"]:
                    if (i in pg): cur["count_con"] += 1
            
            if (cur["state"] == 3 and cur["count_wa"] < 1):
                cur["state"] = 2
            
            if (cur["state"] == 3):
                
                likes = 0
                views = 0
                counter = 0
                cur_date = time.time()
                date_difs = []
                for record in pg["wall_records"]:
                    if ("views" in record):
                        likes += record["likes"]["count"]
                        views += record["views"]["count"]
                        counter += 1
                    date_difs.append(cur_date - record["date"])
                    cur_date = record["date"]
#                 if (len(date_difs) > 0):
#                     cur["wall_median"] = median(date_difs)
#                 else:
#                     cur["wall_median"] = 100000
                dd = [0, 0, 0, 0]
                for i in date_difs:
                    if (i < 7200):
                        dd[0] += 1
                    elif (i < 3600 * 24):
                        dd[1] += 1
                    elif (i < 3600 * 48):
                        dd[2] += 1
                    else:
                        dd[3] += 1
                cur["wall_diffs"] = dd
                if (views == 0):
                    views = 1
                cur["wall_likeview"] = likes / views
                if (cur["count_fr"] == 0):
                    cur["count_fr"] = 10**6
                cur["wall_likefriends"] = likes / cur["count_fr"] / counter * 100
                cur["wall_viewfriends"] = views / cur["count_fr"] / counter
        print("Completed extraction")
        return pd.DataFrame(res)
    
    def save(self, features, file_name = "data_file.csv"):
        df1 = pd.read_csv(file_name)
        df = pd.concat([df1, features])
        df.to_csv(file_name, index = False)
    
    def save_wt_clear(self, features, file_name = "data_file.csv"):
        features.to_csv(file_name, index = False)
    
    def read(self, file_name = "data_file.csv"):
        return pd.read_csv(file_name)

    
    def extract_final(self, df):
        lst = ["screen_name", "count_av", "count_gr", "count_wa", "created_diff", "created_ago", 
                "folow_to_friends", "name_changed", "photo_crop", 
                "state", "wall_diffs", "wall_likefriends", "wall_viewfriends"]
        
        lst = [a for a in lst if a in df]
        res = df[lst]
        return res
    
    def normalize(self, df):
        res = pd.DataFrame()
#         df = df.fillna(value = -1)
#         def nan_ob(val, func):
#             if (val == pd.NaN):
#                 return NaN
#             else:
#                 return func(val)
        res["screen_name"] = df["screen_name"]

        features = {"count_av": lambda x: 0 if (x > 0 and x < 700) else (1 if (x == 0 or x > 1000) else ((x - 700) / 300)),
                    "count_gr": lambda x: 0 if (x < 50) else (1 if x > 150 else (x - 50) / 100),
                    "count_wa": lambda x: 0 if (x != 0) else 1,
                    "created_diff": lambda x: 0 if (x > 604800) else 1,
                    "created_ago": lambda x: 0 if (x > 1209600) else 1,
                    "folow_to_friends": lambda x: 0 if x > 0.1 else 1,
                    "name_changed": lambda x: 1 - x,
                    "photo_crop": lambda x: 0 if (x > 0) else 1,
                    "state": lambda x: x,
                    "wall_diffs": lambda x: 1 if (x[0] == max(x)) else 0,
                    "wall_likefriends": lambda x: 0 if (x > 1.0) else (1 if (x < 0.3) else (((1.0 - x) / 0.7) ** 2)),
                    "wall_viewfriends": lambda x: 0 if (x > 0.5) else (1 if (x < 0.2) else (((0.5 - x) / 0.3) ** 2))
                   }
        
        for feat in features.items():
            if (feat[0] in df):
                res[feat[0]] = df[feat[0]].apply(lambda x: feat[1](x) if isinstance(x, list) or not isnan(x) else 0.5)
            else:
                res[feat[0]] = pd.Series((0.5 for _ in range(len(df))), name=feat[0])
        
        return res
    
    def predict(self, df):
        dfs = []
        dfs.append(pd.concat([df[df["state"] == 0][["screen_name", "state"]], 
                                pd.Series((-1 for _ in range(len(df[df["state"] == 0]))), name="res")], axis=1))
        for i in range(1, 4):
            with open("models/model{}.vkp".format(i), "rb") as fl:
                pred = pk.load(fl)
            df_t = df[df["state"] == i]
            df_t.reset_index(drop=True, inplace=True)
            predicted = pd.Series(pred.predict(df_t.loc[:, df.columns != "screen_name"]), name="res")
            dfs.append(pd.concat([df_t[["screen_name", "state"]], predicted], axis=1))
        return dfs
    


class PredictorV2():
    """loglin predictor"""
    def __init__(self):
        self.coef_ = []
        self.add_coef_ = 0
        self.e_coef_ = 0
        self.edge = 0.41
        return
    
    def single_predictor(self, row):
        
        l = sum([cval * val for cval, val in zip(self.coef_, row)]) + self.add_coef_
        t = [l, l > self.edge]
#         t = (1 / (1 + e ** (-l))) + self.e_coef_
        return t
    
    def predict(self, data):
        res = [self.single_predictor(row[1]) for row in data.iterrows()]
        return res



In [369]:
# Это просто для обкатки
bots = ["id3737274", "zulwey", "id30647801", "id381315484", "mister_bobi", "id178841613", "id339395224"]
semi_bots = ["id315462143", "etoniktonechitaet", "id24020144"]
not_bots = ["vk.com/julia_nicki", "stigmat_na_veke", "id12793494", "dalneva98"]
test = ["id12793494"]
keka = ["0x333", "vk.com/janestep92"]

addresses = bots + semi_bots + good_ids[:10]# + not_bots
# with open("good_ids.txt") as in_file:
#     good_ids = list(map(lambda x: x.rstrip(), in_file.readlines()))
# addresses = good_ids[:700]
# addresses = ["miss__g"]

parser = VkParser(passfile = "passfile.json")
# parser.wall_collect = 0

a = parser.collect_easy_features(parser.format_addresses(addresses))
ex_b = parser.extract_easy_features(a)
ex_c = parser.extract_final(ex_b)
ex_d = parser.normalize(ex_c)

ex_r = parser.predict(ex_d)
res = pd.concat((df for df in ex_r), axis = 0, ignore_index=True)
res


Formating completed
Start Collecting 1000
Main info collected
Dates collected
Photo Collection
Wall Collection
First Records collected
Collection Completed 1000
Started extraction
Completed extraction


,screen_name,state,res
0,miss__g,1,"[0.25, False]"
1,id178841613,2,"[1.03, True]"
2,id3737274,3,"[0.43, True]"
3,zulwey,3,"[0.45, True]"
4,id30647801,3,"[0.362, True]"
5,id381315484,3,"[0.4007260175110692, True]"
6,mister_bobi,3,"[0.5389333333333334, True]"
7,id339395224,3,"[0.64, True]"
8,id315462143,3,"[0.42, True]"
9,etoniktonechitaet,3,"[0.33999999999999997, False]"


In [322]:
ex_r

[Empty DataFrame
 Columns: [screen_name, res]
 Index: []]

In [373]:
pp1 = PredictorV2()
pp1.coef_ = [0.1, 0.13, 0.28, 0, 0.17, 0.2, 0.12, 0.1, 0, 0, 0, 0]
pp1.add_coef_ = 0
pp1.e_coef_ = 0
pp1.edge = 0.35
with open("models/model1.vkp", "wb") as fl:
    pk.dump(pp1, fl)

pp2 = PredictorV2()
pp2.coef_ = [0.03, 0.04, 0.16, 0.2, 0.25, 0.5, 0.1, 0.2, 0, 0, 0, 0]
pp2.add_coef_ = 0
pp2.e_coef_ = 0
pp2.edge = 0.35
with open("models/model2.vkp", "wb") as fl:
    pk.dump(pp2, fl)

pp3 = PredictorV2()
pp3.coef_ = [0.01, 0.01, 0.07, 0.09, 0.11, 0.23, 0.04, 0.09, 0.0, 0.09, 0.07, 0.14]
pp3.add_coef_ = 0
pp3.e_coef_ = 0
pp3.edge = 0.35
with open("models/model3.vkp", "wb") as fl:
    pk.dump(pp3, fl)

In [372]:
def recount(coeffs):
    sm = sum(coeffs)
    for i in range(len(coeffs)):
        coeffs[i] = (coeffs[i] / sm) // 0.01 * 0.01

coef = 

recount(coef)
print(coef)

In [376]:
a = pd.DataFrame([[1, 2]], columns = ["!!!", "!!!!"])
print(a)

   !!!  !!!!
0    1     2


In [309]:
ex_c

,screen_name,count_av,count_gr,count_wa,created_diff,created_ago,folow_to_friends,name_changed,photo_crop,state
0,id3737274,0.0,37,-1,3.608150e+08,3.608153e+08,0.040541,0,1,2
1,zulwey,1379.0,666,-1,3.229934e+08,3.229936e+08,0.017249,1,0,2
2,id30647801,1382.0,90,-1,3.216803e+08,3.216805e+08,0.123195,0,0,2
3,id381315484,34.0,2,-1,8.409827e+07,8.409855e+07,0.199134,0,2,2
4,mister_bobi,834.0,196,-1,2.912585e+08,2.912588e+08,0.018222,1,0,2
5,id178841613,0.0,715,-1,2.132022e+08,2.132025e+08,0.000365,0,0,2
6,id339395224,0.0,760,-1,1.058413e+08,1.058416e+08,0.031088,0,0,2
7,id315462143,174.0,410,-1,1.185608e+08,1.185610e+08,0.247043,0,0,2
8,vomit666,347.0,134,-1,3.319874e+08,3.319877e+08,0.875659,1,1,2
9,etoniktonechitaet,150.0,23,-1,3.318904e+08,3.318907e+08,0.001113,1,2,2


In [379]:
# просто посмотреть чо как
pg_opts = ",".join(["counters", "crop_photo"])
parser.api.users.get(user_ids = "id2", fields = pg_opts)

[{'can_access_closed': False,
  'counters': {'albums': 0,
   'audios': 0,
   'friends': 29,
   'mutual_friends': 0,
   'online_friends': 6,
   'pages': 2,
   'posts': 0,
   'subscriptions': 1,
   'videos': 0},
  'crop_photo': {'crop': {'x': 0.0, 'x2': 100.0, 'y': 5.55, 'y2': 89.88},
   'photo': {'album_id': -6,
    'date': 1482435147,
    'id': 456239317,
    'owner_id': 2,
    'post_id': 34248,
    'sizes': [{'height': 130,
      'type': 'm',
      'url': 'https://pp.userapi.com/c637222/v637222002/1e834/o8-fSYZ32Lc.jpg',
      'width': 73},
     {'height': 231,
      'type': 'o',
      'url': 'https://pp.userapi.com/c637222/v637222002/1e839/wIz7d1Kmgc8.jpg',
      'width': 130},
     {'height': 355,
      'type': 'p',
      'url': 'https://pp.userapi.com/c637222/v637222002/1e83a/TiB0_jG9knc.jpg',
      'width': 200},
     {'height': 568,
      'type': 'q',
      'url': 'https://pp.userapi.com/c637222/v637222002/1e83b/Ps0-w2FLUCI.jpg',
      'width': 320},
     {'height': 900,
      't

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cross_validation import cross_val_score, LeaveOneOut
from sklearn.model_selection import train_test_split

# это для бинарной оценки
def count_metr(my, real, edge)
    counter = 0
    for val1, val2 in zip(my, real):
        my_v = (0 if val1 < edge else 1) 
        if (my_v == val2):
            counter += 1
    return counter / len(my)
    
    
def find_edge(my, real):
    m = 0
    mai = 0
    for i in range(20, 60):
        ii = i / 100
        if (count_metr(my, real, ii) > ma):
            mai = ii
    return mai
        
        
    

data = ex_d
y = ex_d_y
classifier = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(ex_d, ex_d_y, test_size=0.15)

classifier.fit(X_train, y_train)
rs = classifier.predict(X_test)

score, edge = find_edge(rs, y_test)
# t = [l, l > edge]

display(score, edge)

classifier.score(data, y)

loo = cross_validation.LeaveOneOut(len(Y_digits))

# find_edge(classifier.predict(ex_d), ex_d_y)

# это чудо показывает коэфы
classifier.coef_



In [68]:
# получение даты реги, служебное
answ = parser.api.users.get(user_ids = "id99437743")
pg_id = answ[0]["id"]
time.localtime(parser.get_creation_date(pg_id))


time.struct_time(tm_year=2010, tm_mon=9, tm_mday=25, tm_hour=11, tm_min=36, tm_sec=11, tm_wday=5, tm_yday=268, tm_isdst=0)

In [367]:
# построение kr моделей, с графиками и всем таким
from sklearn.kernel_ridge import KernelRidge
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
from plotly import tools
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

import random

init_notebook_mode(connected = True)

a = pd.read_csv("all_id_r.csv")


col_cur = a.iloc[:, 0]
b = a.iloc[:, 53]

keka = a[:, None]

def kek_sc(y1, y2, **kwargs):
    return 1 - sum([0 if (abs(a - b) < 0.15) else  1 for a, b in zip(y1, y2)]) / len(y1)
    

kek_scorer = make_scorer(kek_sc)

kr = GridSearchCV(KernelRidge(kernel = "rbf", gamma = 0.1), scoring = kek_scorer, cv = 5, param_grid = {"gamma": [1, 10, 100, 1000], "alpha": [1, 0.1, 0.01, 0.001]})
kr.fit(keka, b)

# test = kr.predict(keka)
# print(kek_sc(test, b))


print(kr.best_score_)

c = kr.predict(keka)


trace1 = go.Scatter(x = a, y = b, mode = "markers", name = "Source")
trace2 = go.Scatter(x = a, y = c, mode = "lines+markers", name = "Kernel")
fig = tools.make_subplots(rows = 1, cols = 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)

iplot(fig)

In [ ]:
# модуль для выкачивания соответствия vk_id - месяц

import requests
import re
import time

stat = []
pg_id = 1
tek_month = -1
tek_month_count = 1
tek_add = 1

while(True):
    r = requests.get(url = "https://vk.com/foaf.php", params = {"id": pg_id})
    ee = re.search(':created dc:date="([\d, T, \+, \-, :]*)"', r.text)
    if (ee is None):
        pg_id += 10
        continue
    s = ee.group(1)
    tm = (int(i) for i in [s[0:4], s[5:7], s[8:10], s[11:13], s[14:16], s[17:19], 0, 0, 0])
    tm = tuple(tm)
    tm = time.localtime(time.mktime(tm))
    print(tm.tm_year, tm.tm_mon, tek_month_count, tek_add)
    
    if (tek_month_count > 12):
            tek_add *= 10
    
    if (tek_month != tm.tm_mon):
        tek_month_count = 0
        tek_month = tm.tm_mon
        stat.append([str(tm.tm_year) + str(tek_month), pg_id])
    
    tek_month_count += 1
    
    pg_id += tek_add

    
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
init_notebook_mode(connected = True)

x, y = [i[0] for i in stat], [i[1] for i in stat]
trace = go.Scatter(x = x, y = y, mode = "lines")
layout = go.Layout()
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)